In [1]:
import sys


sys.path.append("../src/")

In [2]:
from sennet.core.mmap_arrays import read_mmap_array, create_mmap_array
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import cc3d

In [4]:
# labels_out = cc3d.connected_components(labels_in) # 26-connected
# labels_out = cc3d.dust(
#     labels_in, 
#     threshold=100, 
#     connectivity=26, 
#     in_place=False,
# )

In [7]:
from pathlib import Path


root_path = Path("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/")
mmap_paths = sorted([p for p in root_path.glob("chunk_*")])
for p in mmap_paths:
    print(p)
mmaps = [read_mmap_array(p / "thresholded_prob")for p in mmap_paths]
print([m.shape for m in mmaps])
mmap = np.concatenate([m.data for m in mmaps], axis=0)
# mmap = read_mmap_array("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_02/thresholded_prob")

/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_00
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_01
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_02
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_03
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_04
[[101, 1706, 1510], [101, 1706, 1510], [101, 1706, 1510], [101, 1706, 1510], [97, 1706, 1510]]


In [18]:
connectivity = 26 # only 4,8 (2D) and 26, 18, and 6 (3D) are allowed
labels_out, num_labels = cc3d.connected_components(
    mmap, 
    connectivity=connectivity, 
    # out_file="out.bin",
    return_N=True,
)
labels_out = cc3d.dust(
    labels_out, 
    threshold=1000,
    connectivity=26, 
    in_place=False,
)

In [19]:
!ls -lah

total 269M
drwxrwxr-x  3 clay clay 4.0K Dec 23 19:31 .
drwxrwxr-x 13 clay clay 4.0K Dec 23 18:05 ..
-rw-rw-r--  1 clay clay  36K Dec 19 23:38 2d_mask_proc.ipynb
-rw-rw-r--  1 clay clay  14K Dec 23 19:31 exp_cc3d.ipynb
-rw-rw-r--  1 clay clay 666K Dec 23 18:05 exp_clahe.ipynb
-rw-rw-r--  1 clay clay  72K Dec 22 21:12 exp_eda.ipynb
-rw-rw-r--  1 clay clay  20K Dec 19 23:38 exp_grid_sample.ipynb
-rw-rw-r--  1 clay clay 462K Dec 15 20:59 foreground_splitter.ipynb
-rw-rw-r--  1 clay clay 1.1K Dec 23 19:26 ipykernel_launcher.py.pclprof
drwxrwxr-x  2 clay clay 4.0K Dec 23 18:17 .ipynb_checkpoints
-rw-rw-r--  1 clay clay 993M Dec 23 18:22 out.bin
-rw-rw-r--  1 clay clay 6.2M Dec 23 18:05 test-augmentations.ipynb
-rw-rw-r--  1 clay clay 3.2K Dec 16 14:17 unet.ipynb


In [20]:
unique_labels = np.unique(labels_out)

In [21]:
from tqdm import tqdm


out_mmap = create_mmap_array(
    Path("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/kidney_3_dense/chunk_00/thresholded_prob/"),
    mmap.shape,
    bool,
)
# out_mmap.data[:] = out_image

for label, image in tqdm(cc3d.each(labels_out, binary=True, in_place=True), total=len(unique_labels)):
    out_mmap.data[image] = True
    # out_image = out_image | image
    # out_image = out_image | (image > 0)
out_mmap.data.flush()


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 57/58 [00:12<00:00,  4.72it/s]


In [22]:
from sennet.core.submission_utils import generate_submission_df_from_one_chunked_inference

In [23]:
pred_dir = Path("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/kidney_3_dense")
sub_df = generate_submission_df_from_one_chunked_inference(pred_dir)
sub_df.to_csv(pred_dir / "submission.csv")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.33s/it]


In [24]:
# image

In [25]:
out_mmap.data.sum() / mmap.sum()

0.9784161085606594